# Trabalho Final de Mineração de Dados

## Importando bibliotecas

In [1]:
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
from sklearn import metrics
import pandas as pd
import numpy as np
import nltk
import string
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\demet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\demet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1 - Fase de Preparação dos Dados
---

Carregando o arquivo original de classes.

In [2]:
classes_original = pd.read_excel("dados/SistemaDeClassificacao.xlsx", "Select viw_classificacao_arvore")
classes_original.head()

,,COD_CLASSE,DES_NOME_PREFERIDO,COD_CLASSE_PAI,NUM_NIVEL,CYCLE,TREE,PATH,QTD_FILHOS,DES_NIVEL1,DES_NIVEL2,DES_NIVEL3,DES_NIVEL4
0,1,33254494,Classificação Temática Unificada,NaN,1,0,Classificação Temática Unificada,Classificação Temática Unificada,202,NaN,NaN,NaN,NaN
1,2,33809814,Temas Exclusivos de Pronunciamentos,33254494.0,2,0,-- Temas Exclusivos de Pronunciamentos,Classificação Temática Unificada / Temas Exclu...,22,Temas Exclusivos de Pronunciamentos,NaN,NaN,NaN
2,3,33809634,Meio Ambiente,33254494.0,2,0,-- Meio Ambiente,Classificação Temática Unificada / Meio Ambiente,11,Meio Ambiente,NaN,NaN,NaN
3,4,33809514,"Soberania, Defesa Nacional e Ordem Pública",33254494.0,2,0,"-- Soberania, Defesa Nacional e Ordem Pública","Classificação Temática Unificada / Soberania, ...",7,"Soberania, Defesa Nacional e Ordem Pública",NaN,NaN,NaN
4,5,33808912,Política Social,33254494.0,2,0,-- Política Social,Classificação Temática Unificada / Política So...,37,Política Social,NaN,NaN,NaN


Filtra o arquivo de classes original para exibir apenas aquelas que serão raiz na classificação da ementa.
Essa classes possuem nível 2 na tabela original.

In [3]:
classes_raiz = classes_original.query("NUM_NIVEL == 2").filter(["COD_CLASSE", "DES_NOME_PREFERIDO"])
classes_raiz.rename(columns={"DES_NOME_PREFERIDO": "DES_CLASSE"}, inplace=True)
classes_raiz

,COD_CLASSE,DES_CLASSE
1,33809814,Temas Exclusivos de Pronunciamentos
2,33809634,Meio Ambiente
3,33809514,"Soberania, Defesa Nacional e Ordem Pública"
4,33808912,Política Social
5,33805362,Jurídico
6,33805317,Honorífico
7,33805137,Infraestrutura
8,33769167,Economia e Desenvolvimento
9,33768972,Organização do Estado
10,33685789,Administração Pública


Carrega o dataset contendo as leis que já foram classificadas. Com os dados carregados, cria uma coluna derivada (DES_CLASSE_RAIZ) a partir da árvore de classes (DES_CLASSE_HIERARQUIA), da qual a informação de classe raiz é extraída.

In [4]:
leis_classificadas_original = pd.read_excel("dados/ClassificacaoDeLeisOrdinarias-LeisComplementares-e-DecretosNumerados-Desde1900.xlsx", "Select mvw_u03_prc_doc_tema")
leis_classificadas_original["DES_CLASSE_RAIZ"] = leis_classificadas_original["DES_CLASSE_HIERARQUIA"].apply(lambda hierarquia : hierarquia.split(" / ")[1])
leis_classificadas_original.head()

,,COD_PRC_DOC_TEMA,COD_PROCESSO_DOCUMENTO,COD_CLASSE,DES_CLASSE,DES_CLASSE_HIERARQUIA,DES_CLASSE_RAIZ
0,1,36156374,540837,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
1,2,36158248,540987,33805842,Crédito Extraordinário,Classificação Temática Unificada / Orçamento P...,Orçamento Público
2,3,36158249,540998,33805842,Crédito Extraordinário,Classificação Temática Unificada / Orçamento P...,Orçamento Público
3,4,36156375,541008,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
4,5,36156376,541015,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público


Carrega os dados de todas as leis (classificadas e não classificadas) e aplica as transformações iniciais.

In [5]:
leis_original = pd.read_excel("dados/LeisOrdinarias-LeisComplementare-e-DecretosNumeradosComClassificacaoDesde1900.xlsx", "Select mvw_s01_documento")
leis_original.rename(columns={"DBMS_LOB.SUBSTR(S01.TXT_EMENTA": "TXT_EMENTA"}, inplace=True)
leis_original.drop(columns="   ", inplace=True)
leis_original.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA
0,542244,Lei nº 689 de 20/09/1900,LEI-689-1900-09-20,Autoriza o Governo a recolher em conta corrent...
1,542305,Lei nº 741 de 26/12/1900,LEI-741-1900-12-26,Orça a receita geral da Republica dos Estados ...
2,542241,Lei nº 687 de 14/09/1900,LEI-687-1900-09-14,Fixa as Forças de terra para o exercicio de 1901
3,542412,Lei nº 834 de 30/12/1901,LEI-834-1901-12-30,Fixa a despeza geral da Republica dos Estados ...
4,542387,Lei nº 813 de 23/12/1901,LEI-813-1901-12-23,Fixa a receita geral da Republica dos Estados ...


In [6]:
leis_original.shape[0]

15257

Com os datasets necessários já carregados, cria-se um novo dataset a partir do dataset contendo todas as leis, incluindo-se a informação das classes contidadas nas leis do dataset de leis classificadas.

In [7]:
leis = leis_original.merge(leis_classificadas_original.filter(["COD_PROCESSO_DOCUMENTO","DES_CLASSE_RAIZ"]), left_on="COD_DOCUMENTO", right_on="COD_PROCESSO_DOCUMENTO", how="left")
leis = leis.merge(classes_raiz, left_on="DES_CLASSE_RAIZ", right_on="DES_CLASSE", how="left")
leis.drop(columns=["COD_PROCESSO_DOCUMENTO", "DES_CLASSE_RAIZ"], inplace=True)
leis.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,COD_CLASSE,DES_CLASSE
0,542244,Lei nº 689 de 20/09/1900,LEI-689-1900-09-20,Autoriza o Governo a recolher em conta corrent...,NaN,NaN
1,542305,Lei nº 741 de 26/12/1900,LEI-741-1900-12-26,Orça a receita geral da Republica dos Estados ...,NaN,NaN
2,542241,Lei nº 687 de 14/09/1900,LEI-687-1900-09-14,Fixa as Forças de terra para o exercicio de 1901,NaN,NaN
3,542412,Lei nº 834 de 30/12/1901,LEI-834-1901-12-30,Fixa a despeza geral da Republica dos Estados ...,NaN,NaN
4,542387,Lei nº 813 de 23/12/1901,LEI-813-1901-12-23,Fixa a receita geral da Republica dos Estados ...,NaN,NaN


In [8]:
leis.shape[0]

15671

Limpa os registros duplicados de leis que possuem mais de uma classe folha, mas que tenham mesma classe raiz e exibe as leis que possuem mais de uma classe folha, mas que tenham classes raiz diferentes.

In [9]:
leis.drop_duplicates(inplace=True)
temp = leis[["COD_DOCUMENTO", "COD_CLASSE", "DES_CLASSE"]].groupby("COD_DOCUMENTO")
temp.filter(lambda x: len(x) > 1)

,COD_DOCUMENTO,COD_CLASSE,DES_CLASSE
8844,549877,33805137.0,Infraestrutura
8845,549877,33769167.0,Economia e Desenvolvimento
10839,551926,33805362.0,Jurídico
10840,551926,33808912.0,Política Social
11031,552026,33769167.0,Economia e Desenvolvimento
...,...,...,...
15645,35847286,33805362.0,Jurídico
15648,35957120,33805137.0,Infraestrutura
15649,35957120,33769167.0,Economia e Desenvolvimento
15650,36198674,33769167.0,Economia e Desenvolvimento


Tamanho da base tratada de leis.

In [10]:
leis.shape[0]

15426

Total de classificações feitas em leis.

In [11]:
leis_classificadas = leis.query("not COD_CLASSE.isnull()")
quantidade_classificacoes = leis_classificadas.shape[0]
quantidade_classificacoes

5668

Número distinto de leis classificadas.

In [12]:
len(leis_classificadas["COD_DOCUMENTO"].unique())

5499

Quantidade de leis a classificar.

In [13]:
leis_nao_classificadas = leis.query("COD_CLASSE.isnull()")
leis_nao_classificadas.shape[0]

9758

Verificação da distribuição das classes (em %).

In [14]:
distribuicao_classes = leis_classificadas[["COD_CLASSE", "DES_CLASSE"]].groupby("DES_CLASSE").count() / quantidade_classificacoes * 100
distribuicao_classes.sort_values(by="COD_CLASSE", ascending=False)

,COD_CLASSE
DES_CLASSE,
Orçamento Público,42.925194
Política Social,13.955540
Honorífico,11.009174
Administração Pública,9.262526
Economia e Desenvolvimento,8.874382
Jurídico,6.263232
Infraestrutura,2.875794
Organização do Estado,2.152435
"Soberania, Defesa Nacional e Ordem Pública",1.587862


## 2 - Estruturação do texto
***

Separação em datasets de treino e teste.

In [15]:
X = leis_classificadas.drop(columns=["COD_CLASSE", "DES_CLASSE"])
y = leis_classificadas["DES_CLASSE"]

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

Definição das funções para a tarefa de criação das matrizes de documentos (texto da ementa)

In [16]:
# Divide um texto em tokens utilizando a biblioteca de NLP nltk
def tokenizador(texto):
    texto = texto.lower()
    lista_alfanumerica = []

    for token_valido in nltk.word_tokenize(texto, language="portuguese"):
        # Se for caracter de pontuação ou for stopword
        if token_valido in string.punctuation:
            continue
        # elif token_valido in nltk.corpus.stopwords.words('portuguese'):
        #     continue

        lista_alfanumerica.append(token_valido)

    return lista_alfanumerica

In [17]:
# Combinação de vetores por soma
def combinacao_vetores(palavras_numeros, vocabulario):
    vetor_resultante = np.zeros(300)

    for pn in palavras_numeros:
        try:
            vetor_resultante += vocabulario.get_vector(pn)
        except KeyError:
            if pn.isnumeric():
                pn = "0" * len(pn) # "0, 00, 000, 0000, etc. dependendo do tamanho de caracteres do número
                vetor_resultante += vocabulario.get_vector(pn)
            else:
                vetor_resultante += vocabulario.get_vector("unknown")

    return vetor_resultante

In [18]:
def matriz_vetores(textos, vocabulario):
    x = len(textos)
    y = 300 # número de dimensões do vocabulário skip300
    matriz = np.zeros((x, y))

    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_vetores(palavras_numeros, vocabulario)

    return matriz

Carrega o vocabulário Word Embeddings (Word2Vec - Skipgram) pré-treinado pelo NILC (Núcleo Interinstitucional de Linguística Computacional).

In [19]:
# Carrega vocabulário Skipgram
vocabulario_skipgram = KeyedVectors.load_word2vec_format("dados/skip_s300.txt")

Criação da matriz de documentos do corpus (conjunto de ementas), aqui representada por matrizes por vetores (cada ementa é vetorizada assim como o vocabulário).

In [20]:
matriz_vetores_treino = matriz_vetores(X_treino["TXT_EMENTA"], vocabulario_skipgram)
matriz_vetores_teste = matriz_vetores(X_teste["TXT_EMENTA"], vocabulario_skipgram)

## 3 - Treinamento e predição do modelo de Machine Learning

Treinamento e predição do classificador dummy.

In [21]:
dc = DummyClassifier()
dc.fit(matriz_vetores_treino, y_treino)
classe_prevista_dc = dc.predict(matriz_vetores_teste)

Treinamento e predição do classificador por Regressão Logística.

In [22]:
lr = LogisticRegression(max_iter=3000)
lr.fit(matriz_vetores_treino, y_treino)
classe_prevista_lr = lr.predict(matriz_vetores_teste)

## 4 - Avaliando o modelo

Exibe os relatórios de desempenho do modelo do classificador dummy e do de Regressão Logística no dataset de testes

In [23]:
acuracia_dc = metrics.accuracy_score(y_teste, classe_prevista_dc)
acuracia_lr = metrics.accuracy_score(y_teste, classe_prevista_lr)
print("Acurácia DummyClassifier:",  round(acuracia_dc, 2))
print("Acurácia LinearRegressor:",  round(acuracia_lr, 2))

cr_lr = classification_report(y_teste, classe_prevista_lr)

print(cr_lr)

Acurácia DummyClassifier: 0.43
Acurácia LinearRegressor: 0.77
                                            precision    recall  f1-score   support

                     Administração Pública       0.58      0.65      0.61       158
                Economia e Desenvolvimento       0.58      0.52      0.55       151
                                Honorífico       0.94      0.91      0.92       187
                            Infraestrutura       0.46      0.45      0.45        49
                                  Jurídico       0.48      0.49      0.49       106
                             Meio Ambiente       0.50      0.47      0.49        19
                     Organização do Estado       0.25      0.24      0.25        37
                         Orçamento Público       0.96      0.98      0.97       730
                           Política Social       0.62      0.61      0.62       237
Soberania, Defesa Nacional e Ordem Pública       0.38      0.37      0.38        27

            